In [10]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh, padding, rsa, utils
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import hashlib

parameters = dh.generate_parameters(generator=2, key_size=512, backend=default_backend())

In [11]:
class User: 
    def __init__(self, name): 
        self.name = name
        
        self.identity = parameters.generate_private_key()
        self.identity_public_key = self.identity.public_key()

        self.Ephemeral = parameters.generate_private_key()
        self.Ephemeral_public_key = self.Ephemeral.public_key()

        # Need a way to generate this
        self.Signed_prekey = parameters.generate_private_key()
        self.Signed_prekey_public_key = self.Signed_prekey.public_key()

        self.One_time_prekey = parameters.generate_private_key()
        self.One_time_prekey_public_key = self.One_time_prekey.public_key()
    
        # Need a way to generate this 
        self.Signature = parameters.generate_private_key()
        self.Signate_public_key = self.Signature.public_key()

class server_user: 
    def __init__(self, name, identity, signed_prekey, onetime_prekey, signature):
        self.name = name
        self.identity = identity
        self.signed_prekey = signed_prekey
        self.onetime_prekey = onetime_prekey
        self.signature = signature

In [12]:
Alice = User("Alice")
Bob = User("Bob")

Server = {}
Bob_Server = server_user("Bob", Bob.identity_public_key, Bob.Signed_prekey_public_key, Bob.One_time_prekey_public_key, Bob.Signate_public_key)
Alice_Server = server_user("Alice", Alice.identity_public_key, Alice.Signed_prekey_public_key, Alice.One_time_prekey_public_key, Alice.Signate_public_key)

In [14]:
DH1 = Alice.identity.exchange(Bob.Signed_prekey_public_key)
DH2 = Alice.Ephemeral.exchange(Bob.identity_public_key)
DH3 = Alice.Ephemeral.exchange(Bob.Signed_prekey_public_key)
DH4 = Alice.Ephemeral.exchange(Bob.One_time_prekey_public_key)

# a_shared_key = a_private_key.exchange(b_peer_public_key)
# b_shared_key = b_private_key.exchange(a_peer_public_key)

derived_key_Alice = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(DH1 + DH2 + DH3 + DH4) 

derived_key_Alice

b'X\x80\x95\x16\xd6\x02\xd9~&\x1b\x88\xf8hG\x11I_*k[W\x1c\xacs\x02\xc5\xd8\xfeZV6m'

In [23]:
def getDerivedKey1(User1, User2): 
    DH1 = User1.identity.exchange(User2.Signed_prekey_public_key)
    DH2 = User1.Ephemeral.exchange(User2.identity_public_key)
    DH3 = User1.Ephemeral.exchange(User2.Signed_prekey_public_key)
    DH4 = User1.Ephemeral.exchange(User2.One_time_prekey_public_key)
    derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data',).derive(DH1 + DH2 + DH3 + DH4)
    return derived_key


def getDerivedKey2(User1, User2): 
    DH1 = User2.Signed_prekey.exchange(User1.identity_public_key)
    DH2 = User2.identity.exchange(User1.Ephemeral_public_key)
    DH3 = User2.Signed_prekey.exchange(User1.Ephemeral_public_key)
    DH4 = User2.One_time_prekey.exchange(User1.Ephemeral_public_key)
    derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data',).derive(DH1 + DH2 + DH3 + DH4)
    return derived_key

In [17]:
DH1 = Bob.Signed_prekey.exchange(Alice.identity_public_key)
DH2 = Bob.identity.exchange(Alice.Ephemeral_public_key)
DH3 = Bob.Signed_prekey.exchange(Alice.Ephemeral_public_key)
DH4 = Bob.One_time_prekey.exchange(Alice.Ephemeral_public_key)

derive_key_Bob = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(DH1 + DH2 + DH3 + DH4) 

derive_key_Bob

b'X\x80\x95\x16\xd6\x02\xd9~&\x1b\x88\xf8hG\x11I_*k[W\x1c\xacs\x02\xc5\xd8\xfeZV6m'

b'"f\xb3~\xf5\x8fJ\xd7\x9a\xdb\xb4\xee/\x90\x91\x0b\xb1\x90\x94]\x972\rl\xfd\xd2\xaah\x8b\x8bmo'